In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

OPT = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

inp = "The quick brown fox jumps over the lazy dog"
inp_tokenized = tokenizer(inp, return_tensors="pt")
print(inp_tokenized['input_ids'].size())
print(inp_tokenized)

/home/thomas/env/py3.10-LLMs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
(…)cebook/opt-1.3b/resolve/main/config.json: 100%|████████████████████████████| 653/653 [00:00<00:00, 1.18MB/s]


[2023-10-18 15:02:54,545] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


pytorch_model.bin: 100%|██████████████████████████████████████████████████| 2.63G/2.63G [04:18<00:00, 10.2MB/s]
(…)-1.3b/resolve/main/tokenizer_config.json: 100%|████████████████████████████| 685/685 [00:00<00:00, 3.43MB/s]
(…)acebook/opt-1.3b/resolve/main/vocab.json: 100%|██████████████████████████| 899k/899k [00:00<00:00, 4.53MB/s]
(…)acebook/opt-1.3b/resolve/main/merges.txt: 100%|██████████████████████████| 456k/456k [00:00<00:00, 5.26MB/s]
(…).3b/resolve/main/special_tokens_map.json: 100%|█████████████████████████████| 441/441 [00:00<00:00, 885kB/s]


torch.Size([1, 10])
{'input_ids': tensor([[    2,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


the model and tokenizer using AutoModelForCausalLM and AutoTokenizer, respectively

 the model and tokenizer using AutoModelForCausalLM and AutoTokenizer, respectively

**model's architecture**

In [5]:
print(OPT.model)

OPTModel(
  (decoder): OPTDecoder(
    (embed_tokens): Embedding(50272, 2048, padding_idx=1)
    (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
    (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0-23): 24 x OPTDecoderLayer(
        (self_attn): OPTAttention(
          (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (out_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
        (final_layer_norm): LayerNorm((2048,), eps=1e-05, e

Looking at the embeedding layer

In [4]:
embedded_input = OPT.model.decoder.embed_tokens(inp_tokenized['input_ids'])
print("Layer:\t", OPT.model.decoder.embed_tokens)
print("Size:\t", embedded_input.size())
print("Output:\t", embedded_input)

Layer:	 Embedding(50272, 2048, padding_idx=1)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-0.0407,  0.0519,  0.0574,  ..., -0.0263, -0.0355, -0.0260],
         [-0.0371,  0.0220, -0.0096,  ...,  0.0265, -0.0166, -0.0030],
         [-0.0455, -0.0236, -0.0121,  ...,  0.0043, -0.0166,  0.0193],
         ...,
         [ 0.0007,  0.0267,  0.0257,  ...,  0.0622,  0.0421,  0.0279],
         [-0.0126,  0.0347, -0.0352,  ..., -0.0393, -0.0396, -0.0102],
         [-0.0115,  0.0319,  0.0274,  ..., -0.0472, -0.0059,  0.0341]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<EmbeddingBackward0>)


the positional encoding component utilizes the attention masks to generate a vector that imparts a sense of positioning within the model

In [7]:
embed_pos_input = OPT.model.decoder.embed_positions(inp_tokenized['attention_mask'])
print("Layer:\t", OPT.model.decoder.embed_positions)
print("Size:\t", embed_pos_input.size())
print("Output:\t", embed_pos_input)

Layer:	 OPTLearnedPositionalEmbedding(2050, 2048)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-8.1406e-03, -2.6221e-01,  6.0768e-03,  ...,  1.7273e-02,
          -5.0621e-03, -1.6220e-02],
         [-8.0585e-05,  2.5000e-01, -1.6632e-02,  ..., -1.5419e-02,
          -1.7838e-02,  2.4948e-02],
         [-9.9411e-03, -1.4978e-01,  1.7557e-03,  ...,  3.7117e-03,
          -1.6434e-02, -9.9087e-04],
         ...,
         [ 3.6979e-04, -7.7454e-02,  1.2955e-02,  ...,  3.9330e-03,
          -1.1642e-02,  7.8506e-03],
         [-2.6779e-03, -2.2446e-02, -1.6754e-02,  ..., -1.3142e-03,
          -7.8583e-03,  2.0096e-02],
         [-8.6288e-03,  1.4233e-01, -1.9012e-02,  ..., -1.8463e-02,
          -9.8572e-03,  8.7662e-03]]], device='cuda:0', dtype=torch.float16,
       grad_fn=<EmbeddingBackward0>)


Lastly, the self-attention component! We use the first layer’s self-attention component by indexing through the layers and accessing the .self_attn method.

In [8]:
embed_position_input = embedded_input + embed_pos_input
hidden_states, _, _ = OPT.model.decoder.layers[0].self_attn(embed_position_input)
print("Layer:\t", OPT.model.decoder.layers[0].self_attn)
print("Size:\t", hidden_states.size())
print("Output:\t", hidden_states)

Layer:	 OPTAttention(
  (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
  (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
  (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
  (out_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-0.0119, -0.0110,  0.0056,  ...,  0.0094,  0.0013,  0.0093],
         [-0.0119, -0.0110,  0.0056,  ...,  0.0095,  0.0013,  0.0093],
         [-0.0119, -0.0110,  0.0056,  ...,  0.0095,  0.0013,  0.0093],
         ...,
         [-0.0119, -0.0110,  0.0056,  ...,  0.0095,  0.0013,  0.0093],
         [-0.0119, -0.0110,  0.0056,  ...,  0.0095,  0.0013,  0.0093],
         [-0.0119, -0.0110,  0.0056,  ...,  0.0095,  0.0013,  0.0093]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<MatMul8bitLtBackward>)


## The Encoder-Decoder Architecture

![Encoder-Decoder architecture](TheEncoder-Decorder-Architecture.jpg)

Loading BART model for inspection the architecture

In [9]:
from transformers import AutoModel, AutoTokenizer

BART = AutoModel.from_pretrained("facebook/bart-large")
print(BART)

(…)book/bart-large/resolve/main/config.json: 100%|████████████████████████| 1.63k/1.63k [00:00<00:00, 3.15MB/s]
pytorch_model.bin: 100%|██████████████████████████████████████████████████| 1.02G/1.02G [01:39<00:00, 10.2MB/s]


BartModel(
  (shared): Embedding(50265, 1024, padding_idx=1)
  (encoder): BartEncoder(
    (embed_tokens): Embedding(50265, 1024, padding_idx=1)
    (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
    (layers): ModuleList(
      (0-11): 12 x BartEncoderLayer(
        (self_attn): BartAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layerno

We are already familiar with most of the layers in the BART model. The model is comprised of both encoder and decoder components, with each component consisting of 12 layers. Additionally, The decoder component, in particular, contains an additional **encoder_attn** layer, referred to as cross-attention. The cross-attention component will condition the decoder’s output based on the encoder representations.

We can use the fine-tuned version of this model for summarization using the Transformer’s pipeline functionality.

In [10]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sum = summarizer("""Gaga was best known in the 2010s for pop hits like “Poker Face” and avant-garde experimentation on albums like “Artpop,” and Bennett, a singer who mostly stuck to standards, was in his 80s when the pair met. And yet Bennett and Gaga became fast friends and close collaborators, which they remained until Bennett’s death at 96 on Friday. They recorded two albums together, 2014’s “Cheek to Cheek” and 2021’s “Love for Sale,” which both won Grammys for best traditional pop vocal album.""", min_length=20, max_length=50)
print(sum[0]['summary_text'])

(…)/bart-large-cnn/resolve/main/config.json: 100%|████████████████████████| 1.58k/1.58k [00:00<00:00, 2.96MB/s]
pytorch_model.bin: 100%|██████████████████████████████████████████████████| 1.63G/1.63G [02:39<00:00, 10.2MB/s]
(…)-cnn/resolve/main/generation_config.json: 100%|█████████████████████████████| 363/363 [00:00<00:00, 715kB/s]
(…)k/bart-large-cnn/resolve/main/vocab.json: 100%|██████████████████████████| 899k/899k [00:00<00:00, 3.80MB/s]
(…)k/bart-large-cnn/resolve/main/merges.txt: 100%|██████████████████████████| 456k/456k [00:00<00:00, 11.6MB/s]
(…)rt-large-cnn/resolve/main/tokenizer.json: 100%|████████████████████████| 1.36M/1.36M [00:00<00:00, 3.93MB/s]


Bennett and Gaga became fast friends and close collaborators. They recorded two albums together, 2014's "Cheek to Cheek" and 2021's "Love for Sale"


### The Encoder-Only Architecture

![BERT](Encoder-Only.jpg)

The BERT paper (or an improved variant like RoBERTa) introduced a widely recognized pre-trained model that significantly improved the state-of-the-art scores on numerous NLP tasks. The model undergoes pre-training with two learning objectives:

1. Masked Language Modeling: masking random tokens from the input and attempting to predict them.
2. Next Sentence Prediction: Present sentences in pairs and assess the likelihood of the second sentence in the subsequent sequence of the first sentence.

In [11]:
BERT = AutoModel.from_pretrained("bert-base-uncased")
print(BERT)

(…)rt-base-uncased/resolve/main/config.json: 100%|████████████████████████████| 570/570 [00:00<00:00, 1.11MB/s]
model.safetensors: 100%|████████████████████████████████████████████████████| 440M/440M [00:43<00:00, 10.2MB/s]


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

The following code uses the fine-tuned version of the BERT model for sentiment analysis.

In [12]:
classifier = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
lbl = classifier("""This restaurant is awesome.""")

print(lbl)

(…)cased-sentiment/resolve/main/config.json: 100%|████████████████████████████| 953/953 [00:00<00:00, 5.02MB/s]
pytorch_model.bin: 100%|████████████████████████████████████████████████████| 669M/669M [01:05<00:00, 10.2MB/s]
(…)iment/resolve/main/tokenizer_config.json: 100%|███████████████████████████| 39.0/39.0 [00:00<00:00, 317kB/s]
(…)uncased-sentiment/resolve/main/vocab.txt: 100%|██████████████████████████| 872k/872k [00:00<00:00, 6.89MB/s]
(…)ent/resolve/main/special_tokens_map.json: 100%|█████████████████████████████| 112/112 [00:00<00:00, 226kB/s]


[{'label': '5 stars', 'score': 0.8550480604171753}]


### The Decoder-Only Architecture

![decoder-only architecture](Decoder-Only.jpg)

Large pre-trained models like GPT-4 and LLaMA 2 exhibit the ability to perform tasks such as classification, summarization, translation, etc., by leveraging the appropriate prompt.

The large language models, such as those in the GPT family, undergo pre-training using the Causal Language Modeling objective. This means the model aims to predict the next word, while the attention mechanism can only attend to previous tokens on the left. This implies that the model can solely rely on the previous context to predict the next token and is unable to peek at future tokens, preventing any form of cheating.

In [13]:
gpt2 = AutoModel.from_pretrained("gpt2")
print(gpt2)

(…)ingface.co/gpt2/resolve/main/config.json: 100%|████████████████████████████| 665/665 [00:00<00:00, 3.45MB/s]
model.safetensors: 100%|████████████████████████████████████████████████████| 548M/548M [00:53<00:00, 10.2MB/s]


GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)


The provided code illustrates how the pipeline can be used to incorporate the GPT2 model for text generation. It generates four different alternatives to complete the phrase "This movie was a very.”
Copy

generator = pipeline(model="gpt2")

In [14]:
generator = pipeline(model="gpt2")
output = generator("This movie was a very", do_sample=True, top_p=0.95, num_return_sequences=4, max_new_tokens=50, return_full_text=False)

for item in output:
  print(">", item['generated_text'])

(…)gpt2/resolve/main/generation_config.json: 100%|█████████████████████████████| 124/124 [00:00<00:00, 263kB/s]
(…)gingface.co/gpt2/resolve/main/vocab.json: 100%|████████████████████████| 1.04M/1.04M [00:00<00:00, 4.36MB/s]
(…)gingface.co/gpt2/resolve/main/merges.txt: 100%|██████████████████████████| 456k/456k [00:00<00:00, 2.12MB/s]
(…)face.co/gpt2/resolve/main/tokenizer.json: 100%|████████████████████████| 1.36M/1.36M [00:00<00:00, 10.8MB/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


>  fun and moving experience. The cast and crew were phenomenal. It's amazing how they got here in California.

The cast of The Office went on a run that lasted about 18 hours. During that period of time, all the actors got their
>  much experimental period piece that was very well received in the United States. It's one that I really enjoyed growing up and was very enjoyable. There was so much good, interesting stuff going on.

"I think our story was very strong in
>  popular one. And my husband and I thought it was a great movie because there were so many different genres. And that brought up different things. And we realized, "Well, our experience with this film was that I wasn't happy about it.
>  much part of my life, and when you saw it for the first time, it was kind of like, "Oh, my God. This guy was kind of great, was really good." It was like a great family story." The film really
